In [1]:
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers
from time import time as t
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# loading CIFAR10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# normalizing data
train_images = train_images.reshape((50000, 32, 32, 3)) / 255.0
test_images = test_images.reshape((10000, 32, 32, 3)) / 255.0

In [3]:
# m is the number of training examples
m = train_images.shape[0]
shuffled_indices = np.random.permutation(m)
# shuffling the training data randomely
train_images_shuffled = train_images[shuffled_indices]
train_labels_shuffled = train_labels[shuffled_indices]

# setting 0.20 of the training data to validation set randomly


# one hot encoding of the labels
train_labels_shuffled = to_categorical(train_labels_shuffled, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

## K - Fold Validation

In [4]:
# training function for each fold using data augmentation 
def fold(model, training_set, training_labels, validation_set, validation_set_labels):
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,  
        height_shift_range=0.2,   
        shear_range=0.2,        
        horizontal_flip=True)  

    validation_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(training_set, training_labels, batch_size=32)
    validation_generator = validation_datagen.flow(validation_set, validation_set_labels, batch_size=32)
    history_dataaug_model5 = model.fit_generator(train_generator, validation_data=validation_generator,
                                                 validation_steps=len(training_set) / 32,
                                                 steps_per_epoch=len(training_set) / 32,
                                                 epochs=30, verbose=2)

    score = model.evaluate(validation_set, validation_set_labels, batch_size=128, verbose=0)
    return score[1]

In [5]:
# Using the winner architecture (arch 5) for training the data in each fold and storing the score 
def k_fold_validation(training_set, training_labels, n):
    # scores stores the accuracy of 5 folds
    scores = []

   
    fold_size = math.floor(len(training_set) / n)
    for i in range(n):
        print("---------- Fold", i+1, "----------")
        fold_validation_set = training_set[i * fold_size:i * fold_size + fold_size]
        fold_validation_set_labels = training_labels[i * fold_size:i * fold_size + fold_size]
        fold_training_set = np.delete(training_set, slice(i * fold_size, i * fold_size + fold_size), axis=0)
        fold_training_labels = np.delete(training_labels, slice(i * fold_size, i * fold_size + fold_size), axis=0)
        model_5 = models.Sequential()
        #CNN layers
        #1
        model_5.add(layers.Conv2D(128, (3, 3), activation = 'relu', input_shape = (32, 32, 3)))
        #2
        model_5.add(layers.Conv2D(256, (3, 3), activation = 'relu'))
        #3
        model_5.add(layers.Conv2D(256, (3, 3), activation = 'relu', strides = 2))
        #4
        model_5.add(layers.Conv2D(256, (3, 3), activation = 'relu'))
        #5
        model_5.add(layers.Conv2D(512, (3, 3), activation = 'relu', strides=2))
        #6
        model_5.add(layers.Conv2D(512, (3, 3), activation = 'relu'))

        #Dense layers
        model_5.add(layers.Flatten())
        #Dropout
        model_5.add(layers.Dropout(0.5))
        model_5.add(layers.Dense(256, activation = 'relu'))
        #batch normalization added
        model_5.add(layers.normalization.BatchNormalization())
        model_5.add(layers.Dense(10, activation = 'softmax'))

        model_5.summary()

        model_5.compile(optimizer='rmsprop',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])
        # score of each fold
        s = fold(model_5, fold_training_set, fold_training_labels, fold_validation_set, fold_validation_set_labels)

        scores.append(s)

    print(scores)
    
    # computes the average of 5 accuracies for 5 folds
    scores_average = np.average(scores)
    print("Average scores =", scores_average)


k_fold_validation(train_images_shuffled, train_labels_shuffled, 5)


---------- Fold 1 ----------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 256)       590080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 5, 5, 512)         1180160   
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 512)         2359808   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)      

Epoch 25/30
 - 29s - loss: 0.6928 - acc: 0.7640 - val_loss: 1.0591 - val_acc: 0.6782
Epoch 26/30
 - 28s - loss: 0.6825 - acc: 0.7670 - val_loss: 0.6803 - val_acc: 0.7831
Epoch 27/30
 - 28s - loss: 0.6709 - acc: 0.7691 - val_loss: 0.6496 - val_acc: 0.7880
Epoch 28/30
 - 29s - loss: 0.6566 - acc: 0.7736 - val_loss: 0.6864 - val_acc: 0.7763
Epoch 29/30
 - 29s - loss: 0.6563 - acc: 0.7769 - val_loss: 0.5565 - val_acc: 0.8131
Epoch 30/30
 - 29s - loss: 0.6442 - acc: 0.7804 - val_loss: 0.6150 - val_acc: 0.7893
---------- Fold 3 ----------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 256)       295168    
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 13, 13, 256)       590080    

Epoch 19/30
 - 29s - loss: 0.7637 - acc: 0.7333 - val_loss: 0.6503 - val_acc: 0.7815
Epoch 20/30
 - 29s - loss: 0.7494 - acc: 0.7422 - val_loss: 0.8651 - val_acc: 0.7273
Epoch 21/30
 - 29s - loss: 0.7418 - acc: 0.7435 - val_loss: 0.6663 - val_acc: 0.7756
Epoch 22/30
 - 29s - loss: 0.7175 - acc: 0.7530 - val_loss: 0.5991 - val_acc: 0.7988
Epoch 23/30
 - 29s - loss: 0.7094 - acc: 0.7559 - val_loss: 0.7231 - val_acc: 0.7594
Epoch 24/30
 - 28s - loss: 0.6884 - acc: 0.7650 - val_loss: 0.5798 - val_acc: 0.8072
Epoch 25/30
 - 27s - loss: 0.6906 - acc: 0.7636 - val_loss: 0.6721 - val_acc: 0.7804
Epoch 26/30
 - 27s - loss: 0.6677 - acc: 0.7711 - val_loss: 0.6374 - val_acc: 0.7855
Epoch 27/30
 - 27s - loss: 0.6574 - acc: 0.7749 - val_loss: 0.6263 - val_acc: 0.7935
Epoch 28/30
 - 27s - loss: 0.6511 - acc: 0.7751 - val_loss: 0.5681 - val_acc: 0.8123
Epoch 29/30
 - 29s - loss: 0.6409 - acc: 0.7794 - val_loss: 0.5695 - val_acc: 0.8118
Epoch 30/30
 - 29s - loss: 0.6327 - acc: 0.7834 - val_loss: 0.754

In [6]:
# As it is shown in the result, the average of the accuracies for 5-fold is 77%. In simple hold-out validation 
# we got 80% accuracy which is more than the k-fold result. It means that we chose a good random 
# validation set by chance. If we increase the number of epochs, or shuffle the data differently
#the results would be different. (I did not use the same random shuffling of the data for both
#simple holdout and k-fold techniques.)